# Input Statistical Analysis for all datasets

In [3]:
import numpy as np
import pandas as pd
import spacy
import emoji
from collections import Counter

In [4]:
DATA_DIR = '../data'

### Utility Functions

In [28]:
def remove_emoji(text):
    return emoji.get_emoji_regexp().sub(u'', text)

def slice_dataframe_and_compute_word_frequency(df, slice_cols, slice_vals, text_col, spacy_lang_pkg):
    sliced_df = df.copy()
    for i in range(len(slice_cols)):
        sliced_df = sliced_df[sliced_df[slice_cols[i]] == slice_vals[i]]
    print(f'Found a total of {len(sliced_df)} examples')
    nlp = spacy.load(spacy_lang_pkg)
    text = ' '.join(sliced_df[text_col])
    text = emoji.get_emoji_regexp().sub(r'', text)
    doc = nlp(text)
    words = [token.text for token in doc if not token.is_stop and not token.is_punct and len(token) > 1]
    freqs = Counter(words)
    return freqs

### Spanish (Basile et al.)

In [74]:
df = pd.read_csv(f'{DATA_DIR}/spanish-basile/hateval2019_es_train.csv')

In [75]:
df.head()

,id,text,HS,TR,AG
0,20001,Easyjet quiere duplicar el número de mujeres p...,1,0,0
1,20002,El gobierno debe crear un control estricto de ...,1,0,0
2,20003,Yo veo a mujeres destruidas por acoso laboral ...,0,0,0
3,20004,"— Yo soy respetuoso con los demás, sólamente l...",0,0,0
4,20007,Antonio Caballero y como ser de mal gusto e ig...,0,0,0


#### 1.1 Word Frequecy for Examples that are Hate Speech

In [80]:
freqs = slice_dataframe_and_compute_word_frequency(df, ['HS'], [1], 'text', 'es_core_news_sm')
freqs.most_common()[:10]

Found a total of 1857 examples


[('puta', 418),
 ('perra', 345),
 ('zorra', 216),
 ('Cállate', 122),
 ('callate', 110),
 ('inmigrantes', 102),
 ('mujer', 96),
 ('Callate', 91),
 ('mereces', 82),
 ('cállate', 78)]

#### 1.2 Word Frequecy for Examples that are Hate Speech and Aggressive

In [81]:
freqs = slice_dataframe_and_compute_word_frequency(df, ['HS', 'AG'], [1, 1], 'text', 'es_core_news_sm')
freqs.most_common()[:10]

Found a total of 1502 examples


[('puta', 398),
 ('perra', 322),
 ('zorra', 207),
 ('Cállate', 122),
 ('callate', 109),
 ('Callate', 90),
 ('mereces', 80),
 ('cállate', 78),
 ('mierda', 74),
 ('PUTA', 67)]

#### 1.3 Word Frequecy for Examples that are Hate Speech but not Aggressive

In [82]:
freqs = slice_dataframe_and_compute_word_frequency(df, ['HS', 'AG'], [1, 0], 'text', 'es_core_news_sm')
freqs.most_common()[:10]

Found a total of 355 examples


[('mujer', 54),
 ('inmigrantes', 36),
 ('mujeres', 32),
 ('subsaharianos', 23),
 ('perra', 23),
 ('puta', 20),
 ('España', 17),
 ('país', 17),
 ('papeles', 16),
 ('inmigrante', 15)]

#### 2.  Word Frequecy for Examples that are not Hate Speech

In [83]:
freqs = slice_dataframe_and_compute_word_frequency(df, ['HS'], [0], 'text', 'es_core_news_sm')
freqs.most_common()[:10]

Found a total of 2643 examples


[('puta', 784),
 ('hijo', 237),
 ('acoso', 198),
 ('madre', 149),
 ('perra', 142),
 ('polla', 141),
 ('PUTA', 139),
 ('mierda', 117),
 ('mereces', 112),
 ('violación', 110)]

### Spanish (Pereira et al.)

In [5]:
# Necessary wrangling to fit it into a DataFrame
data = []
with open(f'{DATA_DIR}/spanish-pereira/labeled_corpus_6K.txt', 'r') as f:
    for line in f.readlines():
        curr = line.split(";||;")
        data.append(curr[:-1] + [int(curr[-1][0])])

In [6]:
df = pd.DataFrame(data=np.array(data), columns=['id', 'text', 'HS'])

In [13]:
df['HS'] = df.HS.astype(int)

#### 1. Word Frequecy for Examples that are Hate Speech

In [17]:
freqs = slice_dataframe_and_compute_word_frequency(df, ['HS'], [1], 'text', 'es_core_news_sm')
freqs.most_common()[:10]

                      id                                               text  \
0  id=828025263321657348  Ismael es egocentrico porque se vuelve loca si...   
1  id=828025128797741057  ..ya tardaba en salir quien pronunciase nombre...   
2  id=828025087815274496  (Esto no es un discurso político y razonado, o...   
3  id=828025006626058241  Muy despreciados,siiii,pero todos vestidos de ...   
4  id=828024709761658880  marica explicame porque a veces no te entiendo...   

   HS  
0   0  
1   0  
2   0  
3   1  
4   1  
Found a total of 1567 examples


[('subnormal', 350),
 ('mierda', 144),
 ('puto', 97),
 ('puta', 91),
 ('fascista', 85),
 ('fachas', 82),
 ('facha', 66),
 ('fascistas', 61),
 ('nazi', 44),
 ('retrasado', 42)]

#### 2. Word Frequecy for Examples that are not Hate Speech

In [18]:
freqs = slice_dataframe_and_compute_word_frequency(df, ['HS'], [0], 'text', 'es_core_news_sm')
freqs.most_common()[:10]

                      id                                               text  \
0  id=828025263321657348  Ismael es egocentrico porque se vuelve loca si...   
1  id=828025128797741057  ..ya tardaba en salir quien pronunciase nombre...   
2  id=828025087815274496  (Esto no es un discurso político y razonado, o...   
3  id=828025006626058241  Muy despreciados,siiii,pero todos vestidos de ...   
4  id=828024709761658880  marica explicame porque a veces no te entiendo...   

   HS  
0   0  
1   0  
2   0  
3   1  
4   1  
Found a total of 4433 examples


[('independentista', 501),
 ('subnormal', 306),
 ('fascista', 200),
 ('facha', 196),
 ('fascistas', 157),
 ('negro', 145),
 ('fachas', 135),
 ('mayoría', 124),
 ('género', 121),
 ('gente', 118)]

### French (Ousidhoum et al.)

In [20]:
df = pd.read_csv(f'{DATA_DIR}/french-ousidhoum/fr_dataset.csv')

In [21]:
df.head()

,HITId,tweet,sentiment,directness,annotator_sentiment,target,group
0,1,rt @user personnage ainsi q tte la clique gauc...,abusive,direct,shock,origin,other
1,2,@user @user @user bah oui t'as raison l'autre ...,offensive,indirect,shock,other,individual
2,3,moi j'ai personne à qui m'adresser pour réclam...,hateful,direct,anger_confusion_sadness_fear_disgust,other,women
3,4,@user @user c...est vrai que les anchois sont ...,normal,indirect,indifference,other,other
4,5,eh les renois faut se réveiller la @url,normal,direct,sadness,origin,african_descent


#### 1. Word Frequency for Examples that are not Hate Speech

In [29]:
freqs = slice_dataframe_and_compute_word_frequency(df, ['sentiment'], ['normal'], 'tweet', 'fr_core_news_sm')
freqs.most_common()[:10]

Found a total of 821 examples


[('@user', 970),
 ('@url', 475),
 ('gauchiste', 93),
 ('renois', 79),
 ('attarde', 66),
 ('migrants', 62),
 ('violence', 60),
 ('contre', 53),
 ('arabes', 43),
 ('attardé', 42)]

#### 2. Word Frequency for Examples that are Hate Speech

In [31]:
freqs = slice_dataframe_and_compute_word_frequency(df, ['sentiment'], ['abusive'], 'tweet', 'fr_core_news_sm')
freqs + slice_dataframe_and_compute_word_frequency(df, ['sentiment'], ['offensive'], 'tweet', 'fr_core_news_sm')
freqs + slice_dataframe_and_compute_word_frequency(df, ['sentiment'], ['hateful'], 'tweet', 'fr_core_news_sm')
freqs + slice_dataframe_and_compute_word_frequency(df, ['sentiment'], ['disrespectful'], 'tweet', 'fr_core_news_sm')
freqs + slice_dataframe_and_compute_word_frequency(df, ['sentiment'], ['fearful'], 'tweet', 'fr_core_news_sm')
freqs.most_common()[:10]

Found a total of 594 examples
Found a total of 1336 examples
Found a total of 207 examples
Found a total of 142 examples
Found a total of 236 examples


[('@url', 335),
 ('@user', 239),
 ('mongol', 81),
 ('gauchiste', 77),
 ('renois', 74),
 ('rebeus', 61),
 ('attardé', 56),
 ('rt', 44),
 ('migrants', 35),
 ('sale', 31)]

### Turkish

In [45]:
df = pd.read_csv(f'{DATA_DIR}/turkish/offenseval-tr-training-v1/offenseval-tr-training-v1.tsv', sep='\t')

In [46]:
df.head()

,id,tweet,subtask_a
0,20948,@USER en güzel uyuyan insan ödülü jeon jungkoo...,NOT
1,10134,"@USER Mekanı cennet olsun, saygılar sayın avuk...",NOT
2,23457,Kızlar aranızda kas yığını beylere düşenler ol...,NOT
3,18401,Biraz ders çalışayım. Tembellik ve uyku düşman...,NOT
4,17525,@USER Trezeguet yerine El Sharawy daha iyi olm...,NOT


#### 1. Word Frequency for tweets that are Hate Speech

In [47]:
freqs = slice_dataframe_and_compute_word_frequency(df, ['subtask_a'], ['OFF'], 'tweet', 'en_core_web_sm')
freqs.most_common()[:10]

Found a total of 6046 examples


[('@USER', 5367),
 ('  ', 1380),
 ('bir', 1267),
 ('bu', 1167),
 ('ve', 847),
 ('ne', 718),
 ('de', 688),
 ('da', 660),
 ('için', 546),
 ('gibi', 528)]

#### 2. Word Frequency for tweets that are not Hate Speech

In [ ]:
# This is too long for Spacy. DON'T RUN
freqs = slice_dataframe_and_compute_word_frequency(df, ['subtask_a'], ['NOT'], 'tweet', 'en_core_web_sm')

### Danish

In [48]:
df = pd.read_csv(f'{DATA_DIR}/danish/data/offenseval-da-training-v1.tsv', sep='\t')

In [49]:
df.head()

,id,tweet,subtask_a
0,3131,"Jeg tror det vil være dejlig køligt, men jeg v...",NOT
1,711,Så kommer de nok til at investere i en ny cyke...,NOT
2,2500,Nu er det jo også de Ikea-aber der har lavet s...,OFF
3,2678,"128 Varme emails, er vi enige om at det er sex...",NOT
4,784,"Desværre tyder det på, at amerikanerne er helt...",NOT


#### 1. Word Frequency for tweets that are Hate Speech

In [50]:
freqs = slice_dataframe_and_compute_word_frequency(df, ['subtask_a'], ['OFF'], 'tweet', 'da_core_news_sm')
freqs.most_common()[:10]

Found a total of 384 examples


[('lort', 45),
 ('@USER', 31),
 ('bare', 29),
 ('  ', 24),
 ('godt', 22),
 ('fandme', 22),
 ('når', 19),
 ('folk', 19),
 ('Fuck', 18),
 ('lortet', 15)]

#### 2. Word Frequency for tweets that are not Hate Speech

In [51]:
freqs = slice_dataframe_and_compute_word_frequency(df, ['subtask_a'], ['NOT'], 'tweet', 'da_core_news_sm')
freqs.most_common()[:10]

Found a total of 2576 examples


[('bare', 147),
 ('@USER', 126),
 ('godt', 123),
 ('når', 121),
 ('  ', 117),
 ('Danmark', 100),
 ('se', 82),
 ('helt', 81),
 ('URL', 77),
 ('år', 74)]

### Hindi

In [52]:
df = pd.read_csv(f'{DATA_DIR}/hindi/agr_hi_train.csv')

In [54]:
df.columns = ['id', 'text', 'agr']

In [55]:
freqs = slice_dataframe_and_compute_word_frequency(df, ['agr'], ['OAG'], 'tweet', 'da_core_news_sm')
freqs.most_common()[:10]

,id,text,agr
0,facebook_corpus_msr_386695,Bhai 60sal pehle desh me kya tha pehle pta kro...,CAG
1,facebook_corpus_msr_373389,chutiya friday ko isliye releae krte kyoki wee...,CAG
2,facebook_corpus_msr_917635,जय मोदीराज,CAG
3,facebook_corpus_msr_382517,UPA walo ne bahot kuch kr diya tha desh k liye,CAG
4,facebook_corpus_msr_403274,Pan ko Aadhar se link ki zarurat kuy hai? Supr...,CAG
